# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [175]:
import pandas as pd

treinamento = pd.read_excel('spiderman.xlsx', sheet_name='Treinamento')
teste = pd.read_excel('spiderman.xlsx', sheet_name='Teste')

dfs = [treinamento,teste]

In [176]:
def new_message(tweet):
    tweet = tweet.lower()

    
    space_around = ["?","!"]
    for i in space_around:
        tweet = tweet.replace(i," "+i+" ")

    
    space=[",", ";", "\n", "\xa0","\x00", '"', "(", ")", "“", "”"]
    for k in space:
        tweet = tweet.replace(k," ")
    
    
    remove=["'","…","*","’","‘",'️']
    for j in remove:
        tweet = tweet.replace(j,"")
 

    tweet = tweet.split(" ")

    
    space_link = ["-", ":", ".", "/"]
    
    for i in range(0,len(tweet))[::-1]:
        if tweet[i] == "":
            tweet.pop(i)
        
        elif "http" in tweet[i] or "https" in tweet[i]:
            tweet.pop(i)
        else:
            for l in space_link:
                tweet[i] = tweet[i].replace(l," ")
    
    tweet = " ".join(tweet)
    tweet = tweet.split(" ")
    for i in range(0,len(tweet))[::-1]:
        if tweet[i] == "":
            tweet.pop(i)
    
    return tweet

In [177]:
words = [{"R":[],"I":[]},{"R":[],"I":[]}]
# words_all = [[],[]]
words_dfs = []

for df, index in zip(dfs,range(len(dfs))):
    
    new = []
    
    col_name = df.columns[0]
    
    
    for tweet, df_index in zip(df[col_name],df.index):
        sub_df = df[df.index==df_index]

        new_tweet = new_message(tweet)
        
        words[index][sub_df["Relevância"][df_index]] += new_tweet
        new.append(new_tweet)
        
#         for word in new_tweet:
#             words_all[index].append(word)

    
    R = pd.Series(words[index]["R"]).value_counts().to_frame().rename(columns={0:"R"})
    
    I = pd.Series(words[index]["I"]).value_counts().to_frame().rename(columns={0:"I"})

    joined = R.join(I,how="outer").fillna(0)
            
    joined["Total"] = joined["R"] + joined["I"]
    joined.sort_values(by="Total" ,ascending=False,inplace=True)
    joined["P"] = joined["Total"]/sum(joined["Total"])
    
#     joined["P|R"] = joined["R"]/sum(joined["R"]) # N sei se ta certo :(((((
    
    words_dfs.append(joined)

#     words_all[index]=list(set(words_all[index]))
#     print(len(set(list(R.index)+list(I.index)))==len(words_dfs[-1].index))==len(words_all[index])

# Remover os # das linhas acima para realizar um check simples

In [178]:
words_dfs[0]



# dfs_org = []
# dfs[0].head()
# df
# for item, index in zip(dfs,range(len(dfs))):
    
#     col_name=item.columns[0]
#     dfs_org.append(pd.DataFrame
#                    (data={f"{col_name}" : list(words[index].keys()),
#                           "Value"       : list(words[index].values())}
#                    ).sort_values(by="Value",ascending=False)
#                     .reset_index(drop=True))



R      I  Total         P
spiderman       124.0  118.0  242.0  0.052110
rt              116.0   94.0  210.0  0.045220
the             121.0   75.0  196.0  0.042205
on               93.0   47.0  140.0  0.030146
game            108.0   17.0  125.0  0.026916
new             101.0   15.0  116.0  0.024978
!                18.0   86.0  104.0  0.022394
these            94.0    2.0   96.0  0.020672
are              93.0    3.0   96.0  0.020672
graphics         93.0    0.0   93.0  0.020026
😳                92.0    0.0   92.0  0.019811
ridiculous       92.0    0.0   92.0  0.019811
@juicetoowavie   91.0    0.0   91.0  0.019595
a                 6.0   63.0   69.0  0.014858
to               17.0   44.0   61.0  0.013135
ps4               1.0   53.0   54.0  0.011628
i                 9.0   43.0   52.0  0.011197
and              11.0   38.0   49.0  0.010551
#spiderman        4.0   39.0   43.0  0.009259
spider            3.0   35.0   38.0  0.008183
man               0.0   37.0   37.0  0.007967
is               21.0   13.0   34.0  0.007321
for               3.0   29.0   32.0  0.006891
win               0.0   30.0   30.0  0.006460
that             11.0   17.0   28.0  0.006029
just              6.0   22.0   28.0  0.006029
so               18.0   10.0   28.0  0.006029
you               7.0   20.0   27.0  0.005814
of                4.0   23.0   27.0  0.005814
me                1.0   24.0   25.0  0.005383
...               ...    ...    ...       ...
friends           1.0    0.0    1.0  0.000215
furry             0.0    1.0    1.0  0.000215
gameplay          1.0    0.0    1.0  0.000215
gamespot          1.0    0.0    1.0  0.000215
fifa              0.0    1.0    1.0  0.000215
female            0.0    1.0    1.0  0.000215
echoes            0.0    1.0    1.0  0.000215
feeling           0.0    1.0    1.0  0.000215
eight             0.0    1.0    1.0  0.000215
either            0.0    1.0    1.0  0.000215
elusive           0.0    1.0    1.0  0.000215
emotionally       0.0    1.0    1.0  0.000215
end               1.0    0.0    1.0  0.000215
enix              0.0    1.0    1.0  0.000215
ep2               0.0    1.0    1.0  0.000215
episode           0.0    1.0    1.0  0.000215
est               0.0    1.0    1.0  0.000215
esu               0.0    1.0    1.0  0.000215
european          0.0    1.0    1.0  0.000215
exclusive         1.0    0.0    1.0  0.000215
eyes              0.0    1.0    1.0  0.000215
fair              0.0    1.0    1.0  0.000215
faith             0.0    1.0    1.0  0.000215
fall              0.0    1.0    1.0  0.000215
familiar          0.0    1.0    1.0  0.000215
fans              0.0    1.0    1.0  0.000215
fast              0.0    1.0    1.0  0.000215
fastest           0.0    1.0    1.0  0.000215
fcking            0.0    1.0    1.0  0.000215
🤑                 0.0    1.0    1.0  0.000215

[1013 rows x 4 columns]

In [5]:
# dfs_org[0]["Probability"] = dfs_org[0]["Value"] / len(dfs_org[0])
# dfs_org[0]["ProbabilityR"] = len(dfs_org[0][dfs_org[0].columns[0]].loc[dfs[0]["Relevância"]])
# dfs_org[0]

# R_col=[]
# I_col=[]

# for i in dfs_org[0][dfs_org[0].columns[0]]:    
#     R = 0
#     for j in dfs[0][dfs[0]["Relevância"]=="R"][dfs_org[0].columns[0]]:
#         R+=j.count(i)
    
#     I = 0
#     for j in dfs[0][dfs[0]["Relevância"]=="I"][dfs_org[0].columns[0]]:
#         I+=j.count(i)
    

#     R_col.append(R)
#     I_col.append(I)
# dfs_org[0]["R"]=R_col
# dfs_org[0]["I"]=I_col

In [174]:
%reset -f

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
